In [1]:
import pandas as pd
import numpy as np
def Data_Setup(SectorString=''):
    salary_data = pd.read_csv('Occupation_Salary.csv') 
    workstyles = pd.read_csv('Work Styles.csv')
    workstyles['Data Value'] = workstyles.apply(lambda x: x['Data Value']/(5), axis = 1 )

    workstylespivot = pd.pivot_table(workstyles, values='Data Value', index=['O*NET-SOC Code'],
                    columns=['Element Name'], aggfunc=np.sum)

    jobzones = pd.read_csv('Job Zones.csv') 

    workvalues = pd.read_csv('Work Values.csv')
    workvalues['Score'] = workvalues.apply(lambda x: x['Score']/7, axis = 1 )

    workvaluespivot = pd.pivot_table(workvalues, values='Score', index=['SOC Code'],
                    columns=['Element Name'], aggfunc=np.sum)

    master_table = salary_data[['OCC_CODE', 'Adj Code', 'OCC_TITLE','A_MEAN']]
    master_table['Sector'] = master_table['Adj Code'].str[:2]

    master_table = master_table.merge(jobzones, left_on='OCC_CODE', right_on='O*NET-SOC Code')
    master_table = master_table.merge(workvaluespivot, left_on='OCC_CODE', right_on='SOC Code')
    master_table = master_table.merge(workstylespivot, left_on='OCC_CODE', right_on='O*NET-SOC Code')
    master_table = master_table[master_table['Sector'] == SectorString]
   
    return master_table


In [7]:
import chart_studio
import chart_studio.plotly as py
chart_studio.tools.set_credentials_file(username='jstone11', api_key='VMhFeM8k0oykKfvkY0Ud')
import plotly.express as px


import pandas as pd
mpg = Data_Setup(SectorString='15')

fig = px.scatter(
    mpg,
    x='Achievement',
    y='A_MEAN',
    facet_col='Job Zone',
    hover_name= 'OCC_TITLE',
    category_orders={"Job Zone": ["1. No Prep", "2. Some Prep", "3. Medium Prep", "4. Considerable Prep", "5. Extensive prep"]}

)

py.iplot(fig)